### Урок 4. Парсинг HTML. XPath

Написать приложение, которое собирает основные новости с сайтов mail.ru, lenta.ru, yandex-новости. Для парсинга использовать XPath. Структура данных должна содержать:

- название источника;
- наименование новости;
- ссылку на новость;
- дата публикации.

In [1]:
from lxml import html
import requests
import pandas as pd
from pymongo import MongoClient
from datetime import datetime, timedelta

In [2]:
def get_lenta_news():
    news = []
    url = 'https://m.lenta.ru/parts/news/'
    response = requests.get(url)
    root = html.fromstring(response.text)
    root.make_links_absolute(url)
    articles = root.xpath("//li[@class='parts-page__item']")
    for article in articles:
        source = 'lenta.ru'
        title = str(article.xpath(".//div[@class='card-mini__title']/text()")[0])
        link = str(article.xpath(".//a/@href")[0])
        date = str(article.xpath(".//time/text()")[0])
        date = str(datetime.today().date()) + ' ' + date
        item = {
            'source': source,
            'title': title,
            'link': link,
            'date': date
        }
        news.append(item)

    return news

In [3]:
def get_yandex_news():
    news = []
    url = 'https://yandex.ru/news'
    response = requests.get(url)
    root = html.fromstring(response.text)
    articles = root.xpath("//article")
    for article in articles:
        source = str(article.xpath(".//a[@class='mg-card__source-link']/text()")[0])
        title = str(article.xpath(".//h2/text()")[0])
        link = str(article.xpath(".//a[@class='mg-card__link']/@href")[0])
        date = article.xpath(".//span[@class='mg-card-source__time']/text()")[0].split(' ')
        if date[0] == 'вчера':
            yesterday = datetime.today().date() - timedelta(days=1)
            date = str(yesterday) + ' ' + str(date[2])
        else:    
            date = str(datetime.today().date()) + ' ' + str(date[0])
        item = {
            'source': source,
            'title': title,
            'link': link,
            'date': date
        }
        news.append(item)

    return news

In [4]:
def get_mail_news():
    news = []
    url = 'https://news.mail.ru/'
    response = requests.get(url)
    root = html.fromstring(response.text)
    articles = root.xpath("//div[contains(@class, 'js-ago-wrapper')] | //li[@class='list__item']")
    date_format = '%Y-%m-%dT%H:%M:%S%z'
    
    for article in articles:
        title = str(article.xpath(".//a[contains(@class, 'newsitem__title')]/span/text() | .//a/text() | .//a/span/text()")[0])
        link = str(article.xpath(".//a/@href")[0])
        linkhtml = html.fromstring(requests.get(link).text)
        source = str(linkhtml.xpath("//a[contains(@class, 'breadcrumbs__link')]/span[@class='link__text']/text()")[0])
        date = linkhtml.xpath("//span[@class='note__text breadcrumbs__text js-ago']/@datetime")
        date = datetime.strptime(str(date[0]), date_format).strftime('%Y-%m-%d %H:%M') if date else 'Неизвестно'
        item = {
            'source': source,
            'title': title,
            'link': link,
            'date': date
        }
        news.append(item)

    return news

In [5]:
lentanews = get_lenta_news()
yandexnews = get_yandex_news()
mailnews = get_mail_news()
news = lentanews + yandexnews + mailnews

client = MongoClient('127.0.0.1', 27017)
db = client['news']
news_collection = db.news_collection
news_collection.insert_many(news)

In [6]:
df = pd.DataFrame(news_collection.find()).drop('_id', 1)
df.sample(30)

,source,title,link,date
84,Автоновости дня,Экспорт автомобилей из Южной Кореи упал в сент...,https://yandex.ru/news/story/EHksport_avtomobi...,2021-10-17 15:57
65,ТАСС,Вышел трейлер «Бэтмена» с Робертом Паттинсоном,https://yandex.ru/news/story/Vyshel_trejler_Be...,2021-10-17 15:53
39,ТАСС,Песков: Запад обвиняет Россию в росте цен на г...,https://yandex.ru/news/story/Peskov_Zapad_obvi...,2021-10-17 15:38
78,Planet Today,В Японии с помощью солнечной энергии добыли бо...,https://yandex.ru/news/story/VYAponii_spomoshh...,2021-10-17 15:51
88,Смотрим,Кабо-Верде выдала США венесуэльского дипломата,https://news.mail.ru/incident/48379037/,2021-10-17 00:42
67,Фонтанка.Ру,Концерт Васи Обломова в Петербурге не был согл...,https://yandex.ru/news/story/Koncert_Vasi_Oblo...,2021-10-17 15:34
24,РИА Новости,Володин предложил ввести процедуру отзыва Нобе...,https://yandex.ru/news/story/Volodin_predlozhi...,2021-10-17 16:08
91,МК.RU Санкт-Петербург,Суточная заболеваемость COVID в Петербурге вно...,https://news.mail.ru/society/48374134/,2021-10-16 11:45
82,Автостат,В России начались продажи кроссовера Kia Soren...,https://yandex.ru/news/story/VRossii_nachalis_...,2021-10-17 11:42
104,Известия,Вильфанд предупредил о мокром снеге в европейс...,https://news.mail.ru/society/48381739/,2021-10-17 12:16
